In [1]:
import sys
assert sys.version_info >= (3, 6), "Sonnet 2 requires Python >=3.6"

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tree
import pandas as pd

try:
  import sonnet.v2 as snt
  tf.enable_v2_behavior()
except ImportError:
  import sonnet as snt

print("TensorFlow version {}".format(tf.__version__))
print("Sonnet version {}".format(snt.__version__))

TensorFlow version 2.3.1
Sonnet version 2.0.0


In [4]:
!pwd

/Users/tuckerkj/git/quantumML/python


In [6]:
is_data = pd.read_table('/Users/tuckerkj/Google Drive/Research/QML/data/quc_examples/Tutorial1_TrainPosRealWaveFunction/tfim1d_data.txt', delimiter=' ', usecols=range(10)).values
is_train = is_data[0:7999]
is_test = is_data[8000:9999]

In [7]:
is_train.shape

(7999, 10)

In [8]:
original_dim = is_train.shape[1]

In [329]:
# network parameters
input_shape = (original_dim, )
intermediate_dim = 100
batch_size = 128
latent_dim = 10
epochs = 50

In [99]:
class Encoder(snt.Module):
    def __init__(self, hdims, latent_dim, name=None):
        super(Encoder, self).__init__(name)
        
        self.hidden = []
        for idx, hdim in enumerate(hdims):
            layer_name = 'hidden{}'.format(idx)
            self.hidden.append(snt.Linear(hdim, name=layer_name))
            
        self.z_mean = snt.Linear(latent_dim, name='z_mean')
        self.z_log_var = snt.Linear(latent_dim, name='z_log_var')
        
    def __call__(self, x):
        
        # Run the input through the encoder network
        for hidden_layer in self.hidden:
            x = tf.nn.relu(hidden_layer(x))
        
        # Use the reparameterization trick to calculate the latent variables
        out_mean = self.z_mean(x)
        out_log_var = self.z_log_var(x)
        
        eps = tf.random.normal(out_mean.shape, mean=0.0, stddev=1.0, dtype=tf.dtypes.double)
        out_z = eps*tf.exp(0.5*out_log_var) + out_mean
        
        return {
            'mean': out_mean,
            'log_var': out_log_var,
            'z': out_z
        }

In [100]:
enc = Encoder([intermediate_dim], latent_dim)

In [101]:
output = enc(is_train.astype(float))

In [102]:
output['z']

<tf.Tensor: shape=(7999, 4), dtype=float64, numpy=
array([[-0.84969135, -0.14474033,  0.94266648, -1.33234972],
       [ 1.59825231,  1.27660613, -0.59592931,  0.44321434],
       [-1.16510074,  0.44970454,  0.49334116,  2.55032563],
       ...,
       [-0.70828427,  0.56456185, -0.09377427, -0.41238832],
       [ 1.07908452,  0.89689959,  0.55525758, -0.81118676],
       [ 0.80897817, -0.79185813, -0.88526631, -1.26299141]])>

In [103]:
output['z'].shape

TensorShape([7999, 4])

In [122]:
class Decoder(snt.Module):
    def __init__(self, hdims, vdim, apply_sigmoid=False, name=None):
        super(Decoder, self).__init__(name)
        
        self.apply_sigmoid = apply_sigmoid
        
        self.hidden = []
        for idx, hdim in enumerate(hdims):
            layer_name = 'hidden{}'.format(idx)
            self.hidden.append(snt.Linear(hdim, name=layer_name))
            
        self.visible = snt.Linear(vdim, 'visible')
        
    def __call__(self, x):
        
        for hidden_layer in self.hidden:
            x = tf.nn.relu(hidden_layer(x))
        
        if self.apply_sigmoid:
            output = tf.nn.sigmoid(self.visible(x))
        else:
            output = self.visible(x)
        
        return output

In [222]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.cast(tf.math.log(2. * np.pi), tf.float64)

    return tf.reduce_sum(
        -.5 * ((sample - mean) ** tf.cast(2., tf.float64) * tf.cast(tf.exp(-logvar), tf.float64) + logvar + log2pi),
        axis=raxis)

In [223]:
class VAE(snt.Module):
    def __init__(self, encoder, decoder, name=None):
        super(VAE, self).__init__(name)
        
        self.encoder = encoder
        self.decoder = decoder
                
    def __call__(self, x):
        # Run the encoder and decoder
        encoder_output = self.encoder(x)
        x_recon = self.decoder(encoder_output['z'])
        
        # Compute the loss function
        cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_recon, labels=x)
        log_pxz = -tf.reduce_sum(cross_ent, axis=1)
        log_pz = log_normal_pdf(encoder_output['z'], tf.cast(0.0, tf.float64), tf.cast(0.0, tf.float64))
        log_qzx = log_normal_pdf(encoder_output['z'], encoder_output['mean'], encoder_output['log_var'])
        loss = -tf.reduce_mean(log_pxz + log_pz - log_qzx)
        
        return {
            'x_recon': x_recon,
            'loss': loss
        }

In [224]:
dec = Decoder([intermediate_dim], original_dim, True)

In [225]:
out_vae = dec(out_mean)

In [226]:
out_vae

<tf.Tensor: shape=(7999, 10), dtype=float64, numpy=
array([[0.49805634, 0.56821103, 0.50252593, ..., 0.48917527, 0.49358371,
        0.50441736],
       [0.53601288, 0.5935486 , 0.50996864, ..., 0.58233535, 0.53262848,
        0.60569744],
       [0.5478654 , 0.61765784, 0.51126612, ..., 0.57514374, 0.53254697,
        0.60227551],
       ...,
       [0.52100069, 0.58701677, 0.50656475, ..., 0.51436673, 0.50555004,
        0.52687339],
       [0.48744812, 0.52148054, 0.50447984, ..., 0.4845681 , 0.50866468,
        0.45236271],
       [0.47266674, 0.5267072 , 0.49364781, ..., 0.48846628, 0.51004165,
        0.43183574]])>

In [227]:
vae = VAE(enc, dec)

In [247]:
model_output = vae(is_test.astype(float))

In [248]:
model_output

{'x_recon': <tf.Tensor: shape=(1999, 10), dtype=float64, numpy=
 array([[-2.41030074, -3.95024537, -4.11566051, ..., -2.13874472,
         -0.05711204,  0.65082204],
        [ 1.93131619,  3.65198849,  3.6826388 , ...,  5.53376829,
          4.68197597,  2.42143408],
        [-0.06304897, -1.015196  , -3.01258011, ..., -2.76105152,
         -0.43423799,  0.81827818],
        ...,
        [-0.44084999, -0.44696913, -0.18770231, ...,  2.62859023,
          1.88377851,  0.9240937 ],
        [ 3.05381366,  3.74502294,  1.39615498, ...,  1.20114837,
          0.36089653, -0.14511819],
        [ 0.61598874,  1.84835129,  2.45726687, ...,  0.88366842,
         -0.96806828, -1.01429355]])>,
 'loss': <tf.Tensor: shape=(), dtype=float64, numpy=5.460200439432734>}

## Training

In [230]:
#@title Utility function to show progress bar.
from tqdm import tqdm

num_meas = is_train.shape[0]

def progress_bar(generator):
  return tqdm(
      generator,
      unit='measurements',
      unit_scale=batch_size,
      total=(num_meas // batch_size) * epochs)

In [412]:
# network parameters
learning_rate = 3e-4
input_shape = (original_dim, )
batch_size = 128
epochs = 50

# F = 0.9322
#intermediate_dim = [500, 500]
#latent_dim = 10

# F = 0.9255
intermediate_dim = [100]
latent_dim = 4

optimizer = snt.optimizers.Adam(learning_rate=learning_rate)

enc = Encoder(intermediate_dim, latent_dim)
dec = Decoder(intermediate_dim, original_dim, apply_sigmoid=False)
vae = VAE(enc, dec)

@tf.function
def train_step(data):
    with tf.GradientTape() as tape:
        model_output = vae(tf.cast(data, tf.float64))
    
    trainable_variables = vae.trainable_variables
    grads = tape.gradient(model_output['loss'], trainable_variables)
    optimizer.apply(grads, trainable_variables)
    
    return model_output

In [413]:
# Get data sliced for SGD
train_dataset = (
    tf.data.Dataset.from_tensor_slices(is_train)
    .shuffle(1000)
    .repeat(-1)  # repeat indefinitely
    .batch(batch_size, drop_remainder=True)
    .prefetch(-1))

valid_dataset = (
    tf.data.Dataset.from_tensor_slices(is_test)
    .repeat(1)  # 1 epoch
    .batch(batch_size)
    .prefetch(-1))

In [414]:
# Train
num_training_updates = 10000

train_losses = []
for step_index, data in enumerate(train_dataset):
    train_results = train_step(data)
    train_losses.append(train_results['loss'])
    
    if (step_index + 1) % 100 == 0:
        print('%d loss: %f' % (step_index+1, np.mean(train_losses[-100:])))
        
    if (step_index + 1) % num_training_updates == 0:
        break

100 loss: 7.072758
200 loss: 6.799180
300 loss: 6.474688
400 loss: 6.158727
500 loss: 6.000201
600 loss: 5.908706
700 loss: 5.803221
800 loss: 5.731229
900 loss: 5.668533
1000 loss: 5.634540
1100 loss: 5.607149
1200 loss: 5.581994
1300 loss: 5.556939
1400 loss: 5.531736
1500 loss: 5.526729
1600 loss: 5.499567
1700 loss: 5.487525
1800 loss: 5.480589
1900 loss: 5.467995
2000 loss: 5.465597
2100 loss: 5.459521
2200 loss: 5.451526
2300 loss: 5.450784
2400 loss: 5.449907
2500 loss: 5.447725
2600 loss: 5.445880
2700 loss: 5.427263
2800 loss: 5.443211
2900 loss: 5.432085
3000 loss: 5.427277
3100 loss: 5.434038
3200 loss: 5.428937
3300 loss: 5.435452
3400 loss: 5.426408
3500 loss: 5.425033
3600 loss: 5.435596
3700 loss: 5.418318
3800 loss: 5.420051
3900 loss: 5.422524
4000 loss: 5.408796
4100 loss: 5.417993
4200 loss: 5.399564
4300 loss: 5.428816
4400 loss: 5.420942
4500 loss: 5.409031
4600 loss: 5.412012
4700 loss: 5.410764
4800 loss: 5.413084
4900 loss: 5.406992
5000 loss: 5.411330
5100 loss

# Results

In [415]:
# Look at validation set
model_output = vae(is_test.astype(float))
print(model_output)

{'x_recon': <tf.Tensor: shape=(1999, 10), dtype=float64, numpy=
array([[-1.30727181, -3.54919053, -4.69703729, ..., -5.44847633,
        -4.98149683, -3.04263431],
       [ 2.07257087,  2.37946334,  0.69731315, ...,  4.22436914,
         3.33188312,  1.53229337],
       [-0.63764162, -1.26515629, -1.87520167, ..., -3.63851825,
        -2.23825483, -0.78866909],
       ...,
       [ 0.08981474,  0.32190265,  0.31280718, ...,  2.05090455,
         2.46932876,  1.83831821],
       [ 0.62596565,  1.30627108,  1.68272043, ...,  1.22994615,
        -0.79836201, -1.4194339 ],
       [ 0.94270487,  1.31685063,  1.33196763, ...,  0.66857458,
        -1.80954938, -2.33935259]])>, 'loss': <tf.Tensor: shape=(), dtype=float64, numpy=5.493371116277098>}


In [407]:
def bit_array(a):
    aa = []
    for c in a:
        if c == '0':
            aa.append(0)
        else:
            aa.append(1)
        
    return np.array(aa)

def bin_to_dec(b):
    dec = 0
    for idx, val in enumerate(b):
        dec += val << (len(b) - idx - 1)
        
    return dec

def update_counts(psi, vae, batch_size):
    latent_dim = vae.decoder.hidden[0].input_size
    z = tf.random.normal([batch_size, latent_dim], mean=0.0, stddev=1.0, dtype=tf.dtypes.double)
    output = tf.nn.sigmoid(vae.decoder(z))
    
    #meas = tf.math.round(tf.nn.sigmoid(vae.decoder(z)))
    #meas_int = meas.numpy().astype(int)
    
    eps = tf.random.uniform(output.shape, minval=0, maxval=1, dtype=tf.dtypes.float64)
    meas_int = tf.cast(tf.math.greater_equal(output, eps), tf.int32).numpy()
    
    for ii in range(meas_int.shape[0]):
        idx = bin_to_dec(meas_int[ii,:])
        psi[idx] += 1

def get_psi(vae, num_samples):
    n = vae.encoder.hidden[0].input_size
    psi = np.zeros(2**n)
    batch_size = 1000
    total_samples = 0
    while total_samples < num_samples:
        update_counts(psi, vae, batch_size)
        total_samples = total_samples + batch_size
        
    # Normalize
    psi = np.sqrt(psi*(1.0/float(total_samples)))
    
    return psi

import math
def get_psi_loss(vae, num_samples):
    n = vae.encoder.hidden[0].input_size
    norm = 0
    psi = []
    for d in range(2**n):
        dbin = bit_array(np.binary_repr(d, width=n))
        dbin_input = np.tile(dbin, (num_samples,1))
        model_output = vae(dbin_input.astype(float))
        val = np.exp(-0.5*model_output['loss'])
        psi.append(val)
        norm = norm + val*val
    norm = math.sqrt(norm)
    
    for ii in range(len(psi)):
        psi[ii] = psi[ii]/norm
        
    return np.array(psi)

In [416]:
psi = get_psi(vae, 1000000)

In [384]:
psi = get_psi_loss(vae, 100)

In [417]:
np.dot(psi, psi)

0.9999999999999998

In [418]:
psi

array([0.28247301, 0.1441527 , 0.09196739, ..., 0.09613012, 0.1405169 ,
       0.28005892])

In [419]:
# Save the wave function
np.savetxt('/Users/tuckerkj/Google Drive/Research/QML/data/quc_examples/Tutorial1_TrainPosRealWaveFunction/ld_results/new_vae_psi_4d.txt', psi)


# Scratch

In [34]:
a = ['a','b','c']
for idx, val in enumerate(a):
    print(idx, val)

0 a
1 b
2 c


In [21]:
lin = snt.Linear(intermediate_dim, name='test_lin')

In [27]:
lin(is_train.astype(float))

<tf.Tensor: shape=(7999, 100), dtype=float64, numpy=
array([[-0.46892147, -0.34728384, -0.74378709, ...,  0.29133378,
        -0.26331127,  0.37414721],
       [-0.04339453,  0.07576071, -1.16743727, ...,  0.93418193,
         0.12609772,  0.85569389],
       [ 0.02504136,  0.50216652, -1.25031824, ...,  0.6404713 ,
         0.64565024,  0.74596843],
       ...,
       [ 0.15872905,  0.34844972, -1.76310892, ...,  0.21092538,
         0.73613504,  0.02295605],
       [ 0.31660287,  0.42498682, -0.7207022 , ..., -0.56906129,
         0.97862829, -0.26926874],
       [ 1.17475972,  0.32786622, -1.34069384, ...,  0.04867342,
         0.99408893, -0.87915178]])>